In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format ='retina'
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

In [3]:
import pandas as pd


ModuleNotFoundError: No module named 'pandas'

In [4]:
df = pd.read_csv(r'C:\Users\sumsh\Downloads\HAPS\HAPS\framingham.csv')

NameError: name 'pd' is not defined

In [5]:
df.head(20)

NameError: name 'df' is not defined

In [6]:
df.shape

NameError: name 'df' is not defined

In [ ]:
df.dtypes

In [ ]:
duplicate_df=df[df.duplicated()]
duplicate_df

In [ ]:
df.isna().sum()


In [ ]:
# checking distributions using histograms
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
df.hist(ax = ax)

In [ ]:
# features are correlated with each other 
df_corr = df.corr()
plt.figure(figsize=(12,9))
sns.heatmap(df_corr,annot=True)

In [ ]:
df = df.drop(['education'], axis=1)

In [ ]:
# Checking missing data 
df.isna().sum()

In [ ]:
# Dropping all rows with missing data
df = df.dropna()
df.isna().sum()
df.columns

In [ ]:
# Identifying the features with the most importance for the outcome variable Heart Disease

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# separate independent & dependent variables
X = df.iloc[:,0:14]  #independent columns
y = df.iloc[:,-1]    #target column i.e price range

# apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(11,'Score'))  #print 10 best features

In [ ]:
featureScores = featureScores.sort_values(by='Score', ascending=False)
featureScores

In [ ]:
# visualizing feature selection
plt.figure(figsize=(20,5))
sns.barplot(x='Specs', y='Score', data=featureScores, palette = "GnBu_d")
plt.box(False)
plt.title('Feature importance', fontsize=16)
plt.xlabel('\n Features', fontsize=14)
plt.ylabel('Importance \n', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# selecting the 10 most impactful features for the target variable
features_list = featureScores["Specs"].tolist()[:10]
features_list

In [ ]:
# Create new dataframe with selected features

df = df[['sysBP', 'glucose','age','totChol','cigsPerDay','diaBP','prevalentHyp','diabetes','BPMeds','male','TenYearCHD']]
df.head()

In [ ]:
# Checking correlation again
df_corr = df.corr()
sns.heatmap(df_corr,annot=True)

In [ ]:
# Checking for outliers
df.describe()
sns.pairplot(df)

In [ ]:
# Zooming into cholesterin outliers

sns.boxplot(df.totChol)
outliers = df[(df['totChol'] > 500)] 
outliers

In [ ]:
# Dropping 2 outliers in cholesterin
df = df.drop(df[df.totChol > 599].index)
sns.boxplot(df.totChol)

In [ ]:
df_clean = df

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1)) 

#assign scaler to column:
df_scaled = pd.DataFrame(scaler.fit_transform(df_clean), columns=df_clean.columns)

In [ ]:
df_scaled.describe()
df.describe()

In [ ]:
# clarify what is y and what is x label
y = df_scaled['TenYearCHD']
X = df_scaled.drop(['TenYearCHD'], axis = 1)

# divide train test: 20 % - 80 %
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.80, random_state=29)

In [ ]:
len(X_train)
len(X_test)

In [ ]:
X_test.head()

In [ ]:
sns.barplot(x=df_scaled.TenYearCHD.value_counts().index, y = df_scaled.TenYearCHD.value_counts().values)
plt.box(False)

plt.show()

In [ ]:
shuffled_df = df_scaled.sample(frac=1,random_state=4)

In [ ]:
CHD_df = shuffled_df.loc[shuffled_df['TenYearCHD'] == 1]

In [ ]:
non_CHD_df = shuffled_df.loc[shuffled_df['TenYearCHD'] == 0].sample(n=611,random_state=42)

In [ ]:
normalized_df = pd.concat([CHD_df, non_CHD_df])

In [ ]:
normalized_df.TenYearCHD.value_counts()


In [ ]:
sns.barplot(x=normalized_df.TenYearCHD.value_counts().index, y = normalized_df.TenYearCHD.value_counts().values)

In [ ]:
normalized_df


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 300)

# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, y_train)

# performing predictions on the test dataset
y_pred = clf.predict(X_test)

# metrics are used to find accuracy or error
from sklearn import metrics
print()

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", 100*metrics.accuracy_score(y_test, y_pred))


In [ ]:
print(X_train.values)

In [ ]:
X_train.values


In [ ]:
def start_questionnaire():
    my_predictors = []
    parameters=['sysBP', 'glucose','age','totChol','cigsPerDay','diaBP','prevalentHyp','diabetes','BPMeds','male']
    
    print('Input Patient Information:')
    
    age = input("Patient's age: >>> ") 
    my_predictors.append(age)
    male = input("Patient's gender. male=1, female=0: >>> ") 
    my_predictors.append(male)
    cigsPerDay = input("Patient's smoked cigarettes per day: >>> ") 
    my_predictors.append(cigsPerDay)
    sysBP = input("Patient's systolic blood pressure: >>> ") 
    my_predictors.append(sysBP)
    diaBP = input("Patient's diastolic blood pressure: >>> ")
    my_predictors.append(diaBP)
    totChol = input("Patient's cholesterin level: >>> ") 
    my_predictors.append(totChol)
    prevalentHyp = input("Was Patient hypertensive? Yes=1, No=0 >>> ") 
    my_predictors.append(prevalentHyp)
    diabetes = input("Did Patient have diabetes? Yes=1, No=0 >>> ") 
    my_predictors.append(diabetes)
    glucose = input("What is the Patient's glucose level? >>> ") 
    my_predictors.append(diabetes)
    BPMeds = input("Has Patient been on Blood Pressure Medication? Yes=1, No=0 >>> ")
    my_predictors.append(BPMeds)
    
    my_data = dict(zip(parameters, my_predictors))
    my_df = pd.DataFrame(my_data, index=[0])
    scaler = MinMaxScaler(feature_range=(0,1)) 
   
    # assign scaler to column:
    my_df_scaled = pd.DataFrame(scaler.fit_transform(my_df), columns=my_df.columns)
    my_y_pred = clf.predict(my_df)
    print('\n')
    print('Result:')
    if my_y_pred == 1:
        print("The patient will develop a Heart Disease.")
    if my_y_pred == 0:
        print("The patient will not develop a Heart Disease.")
        
start_questionnaire()

In [ ]:
from joblib import dump

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
dump(clf,"./model.joblib")